In [12]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
!pip install torchsummary


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 10, 3, padding = 'valid') # 28*28   RF = 3*3
        self.norm1 = nn.BatchNorm2d(10)  
        self.conv2 = nn.Conv2d(10, 20, 3, padding = 'valid') #26*26*20  RF = 5*5
        self.norm2 = nn.BatchNorm2d(20)
        self.pool1 = nn.MaxPool2d(2, 2)  #24*24*20  RF = 10*10 
        
        self.conv3 = nn.Conv2d(20, 20, 3, padding = 'valid')  #12*12*20  RF = 12*12
        self.norm3 = nn.BatchNorm2d(20)
        self.conv4 = nn.Conv2d(20,20,3, padding = 'same')   # 10*10*10  RF = 14*14
        self.pool2 = nn.MaxPool2d(2,2)  # 5*5*10     RF = 28*28
        
        self.conv5 = nn.Conv2d(20,10, 1, padding = 'valid')  #5*5*10   RF = 28*28
        self.conv6 = nn.AvgPool2d(5)    # 5*5*10  RF = 32*32
        # self.fc1 = nn.Linear(250,10)
        
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.norm1(self.relu(self.conv1(x)))
        x = self.pool1(self.norm2(self.relu(self.conv2(x))))
        x = self.pool2(self.relu(self.conv4(self.norm3(self.relu(self.conv3(x))))))
        x = self.conv6(self.relu(self.conv5(x)))
        x = x.view(-1, 10)
        # x = self.fc1(x)
        

        return F.log_softmax(x, dim = 1)

In [16]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu") 
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]             100
              ReLU-2           [-1, 10, 26, 26]               0
       BatchNorm2d-3           [-1, 10, 26, 26]              20
            Conv2d-4           [-1, 20, 24, 24]           1,820
              ReLU-5           [-1, 20, 24, 24]               0
       BatchNorm2d-6           [-1, 20, 24, 24]              40
         MaxPool2d-7           [-1, 20, 12, 12]               0
            Conv2d-8           [-1, 20, 10, 10]           3,620
              ReLU-9           [-1, 20, 10, 10]               0
      BatchNorm2d-10           [-1, 20, 10, 10]              40
           Conv2d-11           [-1, 20, 10, 10]           3,620
             ReLU-12           [-1, 20, 10, 10]               0
        MaxPool2d-13             [-1, 20, 5, 5]               0
           Conv2d-14             [-1, 1

In [17]:


torch.manual_seed(1)
batch_size = 256

kwargs = {'num_workers': 6, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [18]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm
    
    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    test_acc.append(100. * correct / len(test_loader.dataset))

In [19]:
model =  Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

EPOCH: 0


Loss=0.38076305389404297 Batch_id=234 Accuracy=55.75: 100%|██████████| 235/235 [00:11<00:00, 20.44it/s]



Test set: Average loss: 0.4702, Accuracy: 8762/10000 (87.62%)

EPOCH: 1


Loss=0.09255564212799072 Batch_id=234 Accuracy=94.31: 100%|██████████| 235/235 [00:11<00:00, 20.17it/s]



Test set: Average loss: 0.1452, Accuracy: 9616/10000 (96.16%)

EPOCH: 2


...


Test set: Average loss: 0.0463, Accuracy: 9861/10000 (98.61%)

EPOCH: 13


Loss=0.010777235962450504 Batch_id=234 Accuracy=99.01: 100%|██████████| 235/235 [00:12<00:00, 19.13it/s]



Test set: Average loss: 0.0499, Accuracy: 9845/10000 (98.45%)

EPOCH: 14


Loss=0.0719601958990097 Batch_id=234 Accuracy=99.03: 100%|██████████| 235/235 [00:11<00:00, 20.56it/s]



Test set: Average loss: 0.0589, Accuracy: 9817/10000 (98.17%)

